# Xception network

This notebook trains the Xception model


After training, model is serialized and uploaded to W&B project.

In [ ]:
import wandb
import pathlib
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from typing import List


def load_data(run) -> List[tf.data.Dataset]:
    """
    Downloads datasets from a wandb artifact and loads them into a list of tf.data.Datasets.
    """

    artifact_name = f"letters_splits_tfds"
    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(
        f"./artifacts/{artifact.name.replace(':', '-')}"
    ).resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()

    # if tf.__version__ minor is less than 10, use
    # tf.data.experimental.load instead of tf.data.Dataset.load

    if int(tf.__version__.split(".")[1]) < 10:
        load_function = tf.data.experimental.load
    else:
        load_function = tf.data.Dataset.load
    
    output_list = []
    for split in ["train", "test", "val"]:
        ds = load_function(str(artifact_dir / split), compression="GZIP")
        output_list.append(ds)
    
    return output_list

def get_number_of_classes(ds: tf.data.Dataset) -> int:
    """
    Returns the number of classes in a dataset.
    """
    labels_iterator= ds.map(lambda x, y: y).as_numpy_iterator()
    labels = np.concatenate(list(labels_iterator))
    return len(np.unique(labels))


def preprocess_dataset(ds: tf.data.Dataset, input_shape: tuple, batch_size: int, cache: bool = True) -> tf.data.Dataset:
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))  # normalize
    # resize from 32x32 to 72x72
    ds = ds.map(lambda x, y: (tf.image.resize(x, input_shape), y))
    ds = ds.unbatch().batch(batch_size)
    if cache:
        ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds


def calculate_model_size_on_disk(path: str) -> int:
    return pathlib.Path(path).stat().st_size


def calculate_model_num_parameters(model: tf.keras.Model) -> int:
    return model.count_params()


def calculate_model_flops() -> str:
    pass


def plot_history(history):
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    epochs = range(1, len(history.history["loss"]) + 1)
    ax.plot(epochs, history.history["accuracy"], label="accuracy")
    ax.plot(epochs, history.history["val_accuracy"], label="val_accuracy")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy")
    ax.legend(loc="lower right")

    plt.show()

def evaluate_model(model, ds_test, model_name):
    """
    Evaluate model test loss, accuracy and other characteristics then log to wandb
    """
    flops = wandb.run.summary["GFLOPs"]
    disk_size = calculate_model_size_on_disk(f"./artifacts/{model_name}.h5")
    num_parameters = calculate_model_num_parameters(model)

    # evaluate model on ds_test and log to wandb
    test_loss, test_acc = model.evaluate(ds_test)

    wandb.log(
        {
            "test loss": test_loss,
            "test accuracy": test_acc,
            "number of parameters": num_parameters,
            "disk size": disk_size,
            "model flops": flops,
        }
    )


def evaluate_diacritics_performance(model, ds_test):
    """
    Evaluate model test loss, accuracy on letters with diacritics then log to wandb
    """
    diacritics = {
        62: "ą",
        63: "ć",
        64: "ę",
        65: "ł",
        66: "ń",
        67: "ó",
        68: "ś",
        69: "ź",
        70: "ż",
        71: "Ą",
        72: "Ć",
        73: "Ę",
        74: "Ł",
        75: "Ń",
        76: "Ó",
        77: "Ś",
        78: "Ź",
        79: "Ż",
    }

    def calculate_batch_size(dataset):
        return next(iter(dataset)).shape[0]

    bs = calculate_batch_size(ds_test)

    # log test accuracy on these classes separately to wandb
    diacritics_acc = {}
    for diacritic_label in diacritics.keys():
        ds_test_diacritic = ds_test.unbatch().filter(lambda x, y: tf.equal(y, diacritic_label)).batch(bs)
        test_loss, test_acc = model.evaluate(ds_test_diacritic)
        diacritics_acc[diacritic_label] = {
            "loss": test_loss,
            "accuracy": test_acc,
            "label": diacritics[diacritic_label],
        }

    wandb.log(diacritics_acc)


In [ ]:
defaults = dict(
    batch_size=32*4,
    epochs=100,    
    optimizer="adam",
    learning_rate=0.001,
    beta1=0.9, 
    beta2=0.999, 
    epsilon=0.1
)

INPUT_SHAPE = [72, 72, 3]
MODEL_NAME = "xception"
run = wandb.init(project="master-thesis", job_type="training", name=MODEL_NAME, config=defaults,)

# hyperparameters

epochs = wandb.config.epochs
bs = wandb.config.batch_size

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Available devices: ", tf.config.list_physical_devices())

In [ ]:
ds_train, ds_test, ds_val = load_data(run)

num_classes = get_number_of_classes(ds_val)

print(f"There are {num_classes} classes")
print(f"Training set has {len(ds_train)} batches")
print(f"Test set has {len(ds_test)} batches")
print(f"Validation set has {len(ds_val)} batches")

ds_train = preprocess_dataset(ds_train, batch_size=bs, input_shape=(INPUT_SHAPE[0], INPUT_SHAPE[1]))
ds_val = preprocess_dataset(ds_val, batch_size=bs, input_shape=(INPUT_SHAPE[0], INPUT_SHAPE[1]))
ds_test = preprocess_dataset(ds_test, batch_size=bs, input_shape=(INPUT_SHAPE[0], INPUT_SHAPE[1]), cache=False)

In [ ]:
from tensorflow.keras.applications.xception import Xception

def get_xception_model(input_shape, num_classes) -> tf.keras.Sequential:
    base_model = Xception(include_top=False, weights=None, input_shape=input_shape)
    model = tf.keras.Sequential()
    model.add(base_model)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))
    return model

In [ ]:
model = get_xception_model(input_shape=INPUT_SHAPE, num_classes=num_classes)

opt = tf.keras.optimizers.get({
    'class_name': wandb.config.optimizer,
    'config': {
        'learning_rate': wandb.config.learning_rate,
        'beta_1': wandb.config.beta1,
        'beta_2': wandb.config.beta2,
        'epsilon': wandb.config.epsilon
    }
})

model.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

wandb_callback = wandb.keras.WandbCallback(
    save_model=False,
    compute_flops=True,
)

# save the best model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"./artifacts/{MODEL_NAME}.h5",
    save_weights_only=False,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
)

history = model.fit(
    ds_train,
    epochs=epochs,
    validation_data=ds_val,
    callbacks=[wandb_callback, checkpoint_callback],
)

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [ ]:
plot_history(history)

In [ ]:
# evaluate model then log to wandb

evaluate_model(model, ds_test, MODEL_NAME)
evaluate_diacritics_performance(model, ds_test)

In [ ]:
# save artifact to wandb
artifact = wandb.Artifact(
    name=MODEL_NAME,
    type="model"
)

# save best model to artifact
artifact.add_file(f"./artifacts/{MODEL_NAME}.h5")
run.log_artifact(artifact)
run.finish()